In [1]:
import duckdb

# 连接到本地数据库文件
con = duckdb.connect(database='tpch-sf1.db', read_only=True)

# 查询数据库中的表名
tables = con.execute("SHOW TABLES").fetchall()
print('Tables:', tables)

# 示例：读取一个表的数据
# result = con.execute('SELECT * FROM lineitem LIMIT 5').fetchdf()
# print(result)

Tables: [('customer',), ('lineitem',), ('nation',), ('orders',), ('part',), ('partsupp',), ('region',), ('supplier',)]


-- SELECT
--     l_orderkey,
--     sum(l_extendedprice * (1 - l_discount)) as revenue,
--     o_orderdate,
--     o_shippriority
-- FROM
--     customer,
--     orders,
--     lineitem
-- WHERE
--     c_mktsegment = 'BUILDING'
--     AND c_custkey = o_custkey
--     AND l_orderkey = o_orderkey
--     AND o_orderdate < date '1995-03-15'
--     AND l_shipdate > date '1995-03-15'
-- GROUP BY
--     l_orderkey,
--     o_orderdate,
--     o_shippriority
-- ORDER BY
--     revenue desc,
--     o_orderdate
-- LIMIT 20;

In [2]:
# 预览 SQL 查询涉及的表
import pandas as pd
from IPython.display import display, Markdown
for table in ['customer', 'orders', 'lineitem']:
    display(Markdown(f'### Preview of table: {table}'))
    df = con.execute(f'SELECT * FROM {table} LIMIT 5').fetchdf()
    display(df)


### Preview of table: customer

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,j5JsirBM9PsCy0O1m,15,25-989-741-2988,711.56,BUILDING,y final requests wake slyly quickly special ac...
1,2,Customer#000000002,487LW1dovn6Q4dMVymKwwLE9OKf3QG,13,23-768-687-3665,121.65,AUTOMOBILE,y carefully regular foxes. slyly regular reque...
2,3,Customer#000000003,fkRGN8nY4pkE,1,11-719-748-3364,7498.12,AUTOMOBILE,fully. carefully silent instructions sleep alo...
3,4,Customer#000000004,4u58h fqkyE,4,14-128-190-5944,2866.83,MACHINERY,sublate. fluffily even instructions are about th
4,5,Customer#000000005,hwBtxkoBF qSW4KrIk5U 2B1AU7H,3,13-750-942-6364,794.47,HOUSEHOLD,equests haggle furiously against the pending p...


### Preview of table: orders

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1996-01-02,5-LOW,Clerk#000000951,0,ly express platelets. deposits acc
1,2,78002,O,46929.18,1996-12-01,1-URGENT,Clerk#000000880,0,ve the furiously fluffy dependencies. carefull...
2,3,123314,F,193846.25,1993-10-14,5-LOW,Clerk#000000955,0,after the asymptotes. instructions cajole aft...
3,4,136777,O,32151.78,1995-10-11,5-LOW,Clerk#000000124,0,st the furiously bold pinto beans. furiously p...
4,5,44485,F,144659.20,1994-07-30,5-LOW,Clerk#000000925,0,onic requests. carefully daring foxes among th...


### Preview of table: lineitem

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,155190,7706,1,17.0,21168.23,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull
1,1,67310,7311,2,36.0,45983.16,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui
2,1,63700,3701,3,8.0,13309.60,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou
3,1,2132,4633,4,28.0,28955.64,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t
4,1,24027,1534,5,24.0,22824.48,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa"


In [3]:
# 查看 SQL 查询涉及的表的行数
for table in ['customer', 'orders', 'lineitem']:
    count = con.execute(f'SELECT COUNT(*) FROM {table}').fetchone()[0]
    print(f'Table {table} has {count} rows.')

Table customer has 150000 rows.
Table orders has 1500000 rows.
Table lineitem has 6001215 rows.


In [4]:
# 导出为 .tbl 格式（管道符分隔，无头行，无引号）
# 使用 FORMAT CSV 并设置 QUOTE '' 来去除引号，HEADER FALSE 去除头行
con.execute("COPY customer TO 'customer.tbl' (FORMAT CSV, DELIMITER '|', HEADER FALSE, QUOTE '')")
con.execute("COPY orders TO 'orders.tbl' (FORMAT CSV, DELIMITER '|', HEADER FALSE, QUOTE '')")
con.execute("COPY lineitem TO 'lineitem.tbl' (FORMAT CSV, DELIMITER '|', HEADER FALSE, QUOTE '')")

print("数据已导出为 .tbl 格式文件（无头行，无引号）：")

数据已导出为 .tbl 格式文件（无头行，无引号）：


In [5]:
# 导出为 .csv 格式（逗号分隔，有头行，无引号）
# 使用 FORMAT CSV 并设置 QUOTE '' 来去除引号，HEADER TRUE 添加头行
con.execute("COPY customer TO 'customer.csv' (FORMAT CSV, DELIMITER ',', HEADER TRUE, QUOTE '')")
con.execute("COPY orders TO 'orders.csv' (FORMAT CSV, DELIMITER ',', HEADER TRUE, QUOTE '')")
con.execute("COPY lineitem TO 'lineitem.csv' (FORMAT CSV, DELIMITER ',', HEADER TRUE, QUOTE '')")

print("数据已导出为 .csv 格式文件（有头行，无引号）：")

数据已导出为 .csv 格式文件（有头行，无引号）：
